Поищем Spark

In [ ]:
import findspark
findspark.init()
findspark.find()

Загрузим необходимые пакеты

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

from pyspark.sql import SparkSession

Создадим Spark-сессию

In [ ]:
spark = SparkSession\
    .builder\
    .appName("TrainValidationSplit")\
    .getOrCreate()

Загрузим данные и проведем первоначальное разбиение на тренировочную и тестовую выборки

In [ ]:
# Prepare training and test data.
data = spark.read.format("libsvm")\
    .load("data/sample_linear_regression_data.txt")
train, test = data.randomSplit([0.9, 0.1], seed=12345)

In [ ]:
data.show()

In [ ]:
lr = LinearRegression(maxIter=10)

Применим ParamGridBuilder для создания сетки параметров для поиска.
TrainValidationSplit попробует все комбинации значений и определит лучшую модель.

In [ ]:
paramGrid = (ParamGridBuilder()
    .addGrid(lr.regParam, [0.1, 0.01]) 
    .addGrid(lr.fitIntercept, [False, True])
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
    .build())

В качестве модели используем линейную регрессию.
Для TrainValidationSplit требуется Estimator, набор Estimator ParamMaps и Evaluator.

In [ ]:
tvs = TrainValidationSplit(estimator=lr,
                           estimatorParamMaps=paramGrid,
                           evaluator=RegressionEvaluator(),
                           # 80% of the data will be used for training, 20% for validation.
                           trainRatio=0.8,
                           parallelism=2)

Запустим TrainValidationSplit и выберем лучший набор параметров.

In [ ]:
model = tvs.fit(train)

Прогноз делается на тестовых данных. Итоговая модель — это модель с комбинацией параметров
которые показали себя лучше всего.

In [ ]:
model.transform(test)\
    .select("features", "label", "prediction")\
    .show()

Остановим Spark

In [ ]:
spark.stop()